# Plotting Temporal changes

I will plot all heights regardless of locations

## Test

One track would be plotted as an example

### Setting

In [ ]:
import glob
import os
import pandas as pd
from pathlib import Path
import h5py
import re
import numpy as np
import matplotlib.pyplot as plt
import datetime
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
##### load files
## set the directory
data_home = Path('/home/jovyan/ICESat_water_level/extraction/download/')
## list them up and check them
files= list(data_home.glob('*.h5'))
## choose a file as an example

##### function of reading alt13
def alt13_to_df(filename, beam):    
    f = h5py.File(filename, 'r')
    f_beam = f[beam]
    lat = f_beam['segment_lat'][:]
    long = f_beam['segment_lon'][:]
    ws = f_beam['ht_water_surf'][:]
    ws_sd = f_beam['stdev_water_surf'][:]
    ws_err = f_beam['err_ht_water_surf'][:]
    ortho = f_beam['ht_ortho'][:]
    wd = f_beam['water_depth'][:]
    alt13_df = pd.DataFrame({'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                            'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                            'WaterD':wd})
    return alt13_df

### We set 'gt2l' as a beam of example for tracking
D_dict={}
error_count=0
for ff in files:
    try:
        D_dict[ff]=alt13_to_df(ff, 'gt2l')
    except KeyError as e:
        ##print(f'file {ff} encountered error {e}')
        error_count += 1
        
### bounds of Tonle Sap Lake
sp_ex = [103.643, 104.667, 12.375, 13.287]

### Plot of one track

##### I need to make a col and draw it repetitively

Function for record the beam, date and RGT, as well as other information.

In [ ]:
### I made a function for stacking the information
def alt13_to_df_beams(filename):    
    f = h5py.File(filename, 'r')
    rgt = str(filename).split("_")[5][0:4]
    cycle = int(str(filename).split("_")[5][4:6])
    version = int(str(filename).split("_")[5][6:8])
    ymd = str(filename).split("_")[4][0:8]
    ymd_trans = datetime.datetime(int(ymd[0:4]),int(ymd[4:6]),int(ymd[6:8]))
    date = ymd_trans.strftime("%Y-%m-%d")
    beam_lst = list(f)[2:-1]
    alt13_df = pd.DataFrame()
    for beam in beam_lst:
        f_beam = f[beam]
        lat = f_beam['segment_lat'][:]
        long = f_beam['segment_lon'][:]
        ws = f_beam['ht_water_surf'][:]
        ws_sd = f_beam['stdev_water_surf'][:]
        ws_err = f_beam['err_ht_water_surf'][:]
        ortho = f_beam['ht_ortho'][:]
        wd = f_beam['water_depth'][:]
        df_beam = pd.DataFrame({'Beam': beam ,'RGT':rgt,'Cycle': cycle, 'Date':date, 'Date_num':int(ymd),'Ver.':version,
                                 'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                                'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                                'WaterD':wd})
        alt13_df = alt13_df.append(df_beam, ignore_index = True)
        
    return alt13_df


In [ ]:
test_beams = alt13_to_df_beams(files[37])

In [ ]:
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(test_beams['Date'],test_beams['SurfaceH'],'.',markersize=0.25, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

## Plot of all tracks with all beams

In [ ]:
a_tracks = pd.DataFrame()
for ff in files:
    a_tracks = a_tracks.append(alt13_to_df_beams(ff), ignore_index = True)
print('done')

In [ ]:
#### Sorting to draw graph
a_tracks.sort_values(by=['Date_num'], inplace=True)

In [ ]:
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.plot(a_tracks['Date'],a_tracks['SurfaceH'],'.',markersize=0.25, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

## Plot the tracks by grouping their locations 

#### Map all tracks with labels

Drawing a just track first.

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(sp_ex, crs=ccrs.PlateCarree())
plt.scatter(a_tracks['Longitude'], a_tracks['Latitude'],s=1)

request = cimgt.Stamen('terrain-background')
ax.add_image(request, 10)
plt.title("Tracks on TSL")

Let's plot water levels according to cycles.

Before it, I will plot the third track, and see the temporal gaps

In [ ]:
#### Select the 3rd track
track_3rd = a_tracks.loc[a_tracks['Cycle'] == 3]

In [ ]:
#### Mapping
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(sp_ex, crs=ccrs.PlateCarree())
plt.scatter(track_3rd['Longitude'], track_3rd['Latitude'],s=1)

request = cimgt.Stamen('terrain-background')
ax.add_image(request, 10)
plt.title("Tracks on TSL")

In [196]:
print(track_3rd['Date_num'].max()-track_3rd['Date_num'].min())

201


It is about 201 day. Too long to see the water changes.

I will see some papers and see how they make a graph.